In [18]:
#import tensorflow as tf
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>") )
import numpy as np
import netCDF4
from netCDF4 import Dataset, num2date
import data_generators
import time
import pygrib
import pickle
# import xarray as xr
# import cfgrib
# import pygrib


fn_mf = "Data/Rain_Data_Nov19/ana_input_1.nc"
fn_rain_Nov = "Data/Rain_Data_Nov19/rr_ens_mean_0.1deg_reg_v20.0e_197901-201907_uk.nc"
fn_rain_Mar = "Data/Rain_Data_Mar20/rr_ens_mean_0.1deg_reg_v20.0e_197901-201907_uk.nc"

#fn  = "/home/u1862646/ATI/BNN/Data/Rain_Data_Nov19/rr_ens_mean_0.1deg_reg_v20.0e_197901-201907_uk.nc" 

def ts_mkr( tss ):
    x = [ time.strftime("%H:%M %d/%m/%y", time.localtime(int(_val))) for _val in tss]
    return x

# Making New netcdf4 file that follows old format

## Getting geopotential data from autoconverted grib file. Since the geopotential data was not correctly copied 

## Netcd4 file making

In [6]:
ds_new = Dataset( 'Data/Rain_Data_Mar20/ana_input.nc','r', format="NETCDF4")

In [11]:
a = ds_new.variables['time'][0].data

In [13]:
a+1

283996801.0

In [4]:
ds_autoconv = Dataset( 'Data/Rain_Data_Mar20/archive/ana_cpdn_combined.nc', "r+", format="NETCDF4" ) #The dataset made by using cdo to convert from grib to cdf, this is missing the geopotential file
ds_old = Dataset( 'Data/Rain_Data_Nov19/ana_input.nc', "r", format="NETCDF4" ) #The dataset used in nov19 folder for model field data
ds_new = Dataset( 'Data/Rain_Data_Mar20/ana_input_new.nc','w', format="NETCDF4")

print( "AutoConv keys:{}".format(ds_autoconv.variables.keys()))
print( "Old keys:{}".format(ds_old.variables.keys()))

#Creating new Dimensions
ds_new.createDimension('time', 59900)
ds_new.createDimension('latitude', 20)
ds_new.createDimension('longitude', 21)

#formatting all variables to conform with old dataset

#geopotential
    # Adding new Geopotential Data
geo_data = pickle.load(open("Data/Rain_Data_Mar20/archive/geopotential.np","rb")  )
ds_new.createVariable('geopotential','float64', dimensions=('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36 )
ds_new.variables['geopotential'][:] = geo_data

ds_new.variables['geopotential'].standard_name = "geopotential"
ds_new.variables['geopotential'].long_name = "geopotential"
ds_new.variables['geopotential'].units = "m2 s-2"
ds_new.variables['geopotential'].grid_mapping = "latitude_longitude"

del geo_data

#y_wind
ds_new.createVariable('y_wind', 'float64', dimensions=('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36  )
ds_new.variables['y_wind'].standard_name = "y_wind"
ds_new.variables['y_wind'].long_name = "y_wind"
ds_new.variables['y_wind'].units = "m s-1"
ds_new.variables['y_wind'][:] =  ds_autoconv.variables['v'][:]

#x_wind
ds_new.createVariable('x_wind', 'float64', dimensions=('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36 )
ds_new.variables['x_wind'].standard_name = "x_wind"
ds_new.variables['x_wind'].long_name = "x_wind"
ds_new.variables['x_wind'].units = "m s-1"
ds_new.variables['x_wind'][:] =  ds_autoconv.variables['u'][:]

#time
ds_new.createVariable('time', 'float64', dimensions=('time'), fill_value=9.969209968386869e+36)
ds_new.variables['time'].axis = "T"
ds_new.variables['time'].units ="hours since 1970-01-01 00:00:00"
ds_new.variables['time'].standard_name = "time"
ds_new.variables['time'].calendar = "gregorian"
ds_new.variables['time'][:] =  ds_autoconv.variables['time'][:]

#latitude
ds_new.createVariable('latitude', 'float64', dimensions=('latitude'), fill_value=9.969209968386869e+36 )
ds_new.variables['latitude'].axis = "Y"
ds_new.variables['latitude'].units = "Degrees North"
ds_new.variables['latitude'].standard_name = "latitude"
ds_new.variables['latitude'][:] = ds_autoconv.variables['latitude'][:]

#longitude
ds_new.createVariable('longitude', 'float64', dimensions=('longitude'), fill_value=9.969209968386869e+36)
ds_new.variables['longitude'].axis = "Y"
ds_new.variables['longitude'].units = "Degrees East"
ds_new.variables['longitude'].standard_name = "longitude"
ds_new.variables['longitude'][:] = ds_autoconv.variables['longitude'][:]


#unknown_local_param_137_128 (total coloumn water vapour)
ds_new.createVariable('unknown_local_param_137_128', 'float64', dimensions=('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36)
ds_new.variables['unknown_local_param_137_128'].standard_name = "total coloumn water vapour" 
ds_new.variables['unknown_local_param_137_128'][:] = ds_autoconv.variables['tcwv'][:]


#unknown_local_param_133_128 (specific humidity)
ds_new.createVariable('unknown_local_param_133_128', 'float64', dimensions= ('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36 )
ds_new.variables['unknown_local_param_133_128'].standard_name = "specific humidity"
ds_new.variables['unknown_local_param_133_128'][:] = ds_autoconv.variables['q'][:]

#air_temperature
ds_new.createVariable('air_temperature','float64', dimensions= ('time', 'latitude', 'longitude'), fill_value=9.969209968386869e+36 )
ds_new.variables['air_temperature'].units = "K"
ds_new.variables['air_temperature'][:] = ds_autoconv.variables['t'][:]

ds_new.close()

AutoConv keys:dict_keys(['number', 'time', 'step', 'surface', 'latitude', 'longitude', 'valid_time', 'tcwv', 'isobaricInhPa', 't', 'q', 'u', 'v'])
Old keys:dict_keys(['unknown_local_param_137_128', 'latitude_longitude', 'time', 'latitude', 'longitude', 'forecast_period', 'originating_centre', 'unknown_local_param_133_128', 'pressure', 'air_temperature', 'geopotential', 'pressure_0', 'x_wind', 'y_wind'])


# Data Inspection

In [ ]:
#ATI Means for variables
vars_for_feature = ['unknown_local_param_137_128', 'unknown_local_param_133_128', 'air_temperature', 'geopotential', 'x_wind', 'y_wind' ]
with Dataset(fn_mf, "r", format="NETCDF4") as f:
    for idx, name in enumerate(vars_for_feature):
        _dat = f.variables[name][:]
        print( name)
        print("mean","\t",_dat.mean() )
        print("max","\t",_dat.max() )
        print("min","\t",_dat.min() )
        print("\n")
with Dataset(fn_rain, "r", format="NETCDF4") as f:
    _dat = f.variables["rr"][:]
    print( "rain")
    print("mean","\t",_dat.mean() )
    print("max","\t",_dat.max() )
    print("min","\t",_dat.min() )
    print("\n")

In [ ]:
with Dataset(fn_mf, "r", format="NETCDF4") as f1:
    _data_mf = f1.variables["time"][:]
    starting_time_mf = _data_mf[0]
#     a = f.variables['longitude'][:]
#     b = f.variables['latitude'][:]

with Dataset(fn_rain, "r", format="NETCDF4") as f2:
    _data_r = f2.variables["time"][:]
    starting_time_r = _data_r[0]
#     a = f.variables['longitude'][:]
#     b = f.variables['latitude'][:]

In [ ]:
print(starting_time_mf)
print(starting_time_r)

#"""Rain starts 10592 days after 1950-01-01:00:00:00 """
#"""MF starts hours since 1979-01-01:00:"""

In [ ]:
_data_mf[0]2

In [ ]:
np.alltrue((_data_r[1:] - _data_r[:-1])==1)

In [ ]:
time_length = 4
print( ts_mkr(_data_mf[:time_length*4]) )
print( ts_mkr(_data_r[:time_length]) )

In [ ]:
_data_r[:100]

In [ ]:
_data_mf[:8]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def heatmap2d(arr: np.ndarray):
    plt.imshow(arr, cmap='viridis')
    plt.colorbar()
    plt.show()
    
heatmap2d(_data[1500, :,:])

In [ ]:
start_idx = 10500
_data[start_idx:start_idx+100,70,102]
_data[start_idx,70,102]

In [ ]:
ds_rain.find_idx_of_city("London")

In [ ]:
ds_mf = data_generators.Generator_mf(fn = fn_mf)
#ds_gen = ds_mf()
#ds_gen_tf = tf.data.Dataset.from_generator( ds_mf, output_types=(tf.float32, tf.bool) )
#ds_gen_tf = ds_gen_tf = ds_gen_tf.batch(100)
#ds_gen_tf_iter = iter(ds_gen_tf)


In [ ]:
a = next(ds_gen)

In [ ]:
a[1].shape


In [ ]:
next(ds_gen_tf_iter)

In [ ]:
ds_rain = data_generators.Generator_rain(fn = fn_rain)
ds_rain_gen = ds_rain()
ds_rain_gen_tf = tf.data.Dataset.from_generator( ds_rain, output_types=(tf.float32, tf.bool) )
ds_rain_gen_tf = ds_gen_tf = ds_rain_gen_tf.batch(100)
ds_rain_gen_tf_iter = iter(ds_gen_tf)

# Testing Shermans new data extraction methods


## Rain 

In [ ]:
dataset_rain = Dataset(fn_rain, "r", format="NETCDF4")
data_rain = dataset_rain.variables
print(dataset_rain.__doc__)

In [ ]:
type(dataset_rain)

In [ ]:
#data is an ordered dict
#loop through the keys and print it
print("The name of the variables are:")
for key in data_rain.keys():
    print(key)
print()


In [ ]:
#print each item in dat
print("For each of the variables, here are the properties:\n")
for key, value in data_rain.items():
    print("Variable: " + key)
    print(value)
    print()

In [ ]:
a = data_rain['rr'][1000].mask
np.count_nonzero(a)/np.size(a)

In [ ]:
np.argwhere(a=False)

In [ ]:
np.asarray( data['rr'] ).shape

In [ ]:
data['time'][:20]

## model fields

In [ ]:
mf_dataset = Dataset(fn_mf, "r", format="NETCDF4")
mf_data = mf_dataset.variables

In [ ]:
#data is an ordered dict
#loop through the keys and print it
print("The name of the variables are:")
for key in mf_data.keys():
    print(key)
print()


In [ ]:
#print each item in dat
print("For each of the variables, here are the properties:\n")
for key, value in mf_data.items():
    print("Variable: " + key)
    print(value)
    print()

In [ ]:
mf_data['air_temperature'].shape

# Testing Data Generators

In [ ]:
# import tensorflow as tf
# import netCDF4
# from netCDF4 import Dataset
# import gdal
# import numpy as np

#TODO:(akanni-ade): add ability to return long/lat variable  implement long/lat

"""
    Example of how to use
    import Generators

    rr_ens file 
    _filename = "Data/Rain_Data/rr_ens_mean_0.1deg_reg_v20.0e_197901-201907_djf_uk.nc"
    rain_gen = Generator_rain(_filename, all_at_once=True)
    data = next(iter(grib_gen))

    Grib Files
    _filename = 'Data/Rain_Data/ana_coarse.grib'
    grib_gen = Generators.Generator_grib(fn=_filename, all_at_once=True)
    data = next(iter(grib_gen))

    Grib Files Location:
    _filename = 'Data/Rain_Data/ana_coarse.grib'
    grib_gen = Generators.Generator_grib(fn=_filename, all_at_once=True)
    arr_long, arr_lat = grib_gen.locaiton()
    #now say you are investingating the datum x = data[15,125]
    #   to get the longitude and latitude you must do
    long, lat = arr_long(15,125), arr_lat(15,125)


"""
class Generator():
    
    def __init__(self, fn = "", all_at_once=False, train_size=0.75, channel=None ):
        self.generator = None
        self.all_at_once = all_at_once
        self.fn = fn
        self.channel = channel
    
    def yield_all(self):
        pass

    def yield_iter(self):
        pass

    def long_lat(self):
        pass

    def __call__(self,x=0):
        if(self.all_at_once):
            return self.yield_all()
        else:
            return self.yield_iter(x)
         
class Generator_mf(Generator):
    """
        Creates a generator for the model_fields_data
    
        :param all_at_once: whether to return all data, or return data in batches

        :param channel: the desired channel of information in the grib file
            Default None, then concatenate all channels together and return
            If an integer return this band
    """

    def __init__(self, **generator_params):

        super(Generator_mf, self).__init__(**generator_params)

        self.vars_for_feature = ['unknown_local_param_137_128', 'unknown_local_param_133_128', 'air_temperature', 'geopotential', 'x_wind', 'y_wind' ]
        self.channel_count = len(self.vars_for_feature) 
        

    def yield_all(self):
        raise NotImplementedError
    
    def yield_iter(self):
        with Dataset(self.fn, "r", format="NETCDF4") as f:
            for tuple_mfs in zip( *[f.variables[var_name] for var_name in self.vars_for_feature ] ):
                
                list_datamask = [ (np.ma.getdata(_mar),np.ma.getmask(_mar) ) for _mar in tuple_mfs ]
                
                _data, _masks= list( zip (*list_datamask ) )
                
                
                stacked_data = np.stack(_data, axis=-1 )
                stacked_masks = np.stack(_masks, axis=-1 )
                
                yield stacked_data, stacked_masks
            
            
        
    def location(self):
        """
        Returns a 2 1D arrays
            arr_long: Longitudes
            arr_lat: Latitdues
        Example of how to use:


        """
        raise NotImplementedError
        
class Generator_rain2(Generator):
    def __init__(self, **generator_params):
        super(Generator_rain2, self).__init__(**generator_params)

    def yield_all(self,start_idx=0):
        with Dataset(self.fn, "r", format="NETCDF4") as f:
            _data = f.variables['rr'][start_idx:]
            yield np.ma.getdata(_data), np.ma.getmask(_data)   
            
    def yield_iter(self,start_idx=0):
        f = Dataset(self.fn, "r", format="NETCDF4")
        #with Dataset(self.fn, "r", format="NETCDF4") as f:
            #for chunk in f.variables['rr'][start_idx:]:
        
        #for chunk in f.variables['rr'][start_idx:]:
        for chunk in f.variables['rr'][start_idx:]:
            yield np.ma.getdata(chunk), np.ma.getmask(chunk)

## rain genenerator

In [ ]:
#rain_gen = Generator_rain(fn=fn_rain, all_at_once=False)()
rain_gen2 = Generator_rain2(fn=fn,all_at_once=False)()
data = next(iter(rain_gen2))
data

In [ ]:
fn_rain

In [ ]:
ds_tar = tf.data.Dataset.from_generator(lambda : Generator_rain2(fn=fn, all_at_once=False)(0), output_types= ( tf.float32, tf.bool) )

In [ ]:
next(iter(ds_tar) )

## model feild generator

In [ ]:
mf_gen = Generator_mf(fn=fn_mf, all_at_once=False)()
data = next(iter(mf_gen))

In [ ]:
data[0].shape

In [ ]:
mask.shape

# Fixing mismatch in size of Feature and Target in Nov Data

In [16]:
rain_lat = data_rain['latitude'] #100
rain_lon = data_rain['longitude'] #140

mf_latlon = mf_data['latitude_longitude']
mf_lat = mf_data['latitude'] #103
mf_lon = mf_data['longitude'] #144

NameError: name 'data_rain' is not defined

In [ ]:
rain_lat[:]

In [ ]:
mf_lat[::-1]

In [ ]:
#Latitude transformation 
mf_lat[::-1][1:-2] - rain_lat[:]

In [ ]:
mf_lat[-2:1:-1] - rain_lat[:]

In [ ]:
rain_lon[:]

In [ ]:
mf_lon[:]

In [ ]:
rain_lon[:] - mf_lon[2:-2]

# Fixing mismatch in size/location of Feature and Target in March Data

In [21]:
ds_mf_new = Dataset( 'Data/Rain_Data_Mar20/ana_input.nc','r', format="NETCDF4")
ds_rain = Dataset(fn_rain_Mar, "r", format="NETCDF4")

In [22]:
rain_lat = ds_rain.variables['latitude'] #100
rain_lon = ds_rain.variables['longitude'] #140

mf_lat = ds_mf_new.variables['latitude'] #20 --> 18
mf_lon = ds_mf_new.variables['longitude'] #21 --> 18

## Latitude

In [24]:
rain_lat[::6]

masked_array(data=[49.04985973, 49.6498597 , 50.24985968, 50.84985965,
                   51.44985963, 52.0498596 , 52.64985958, 53.24985955,
                   53.84985953, 54.4498595 , 55.04985948, 55.64985945,
                   56.24985943, 56.8498594 , 57.44985938, 58.04985935,
                   58.64985933],
             mask=False,
       fill_value=1e+20)

In [28]:
mf_lat[-2:1:-1]

masked_array(data=[49.44363158, 49.99926316, 50.55489474, 51.11052632,
                   51.66615789, 52.22178947, 52.77742105, 53.33305263,
                   53.88868421, 54.44431579, 54.99994737, 55.55557895,
                   56.11121053, 56.66684211, 57.22247368, 57.77810526,
                   58.33373684],
             mask=False,
       fill_value=1e+20)

In [27]:
#Latitude transformation 
rain_lat[::6]-mf_lat[-2:1:-1]

masked_array(data=[-0.39377185, -0.34940346, -0.30503506, -0.26066666,
                   -0.21629827, -0.17192987, -0.12756148, -0.08319308,
                   -0.03882468,  0.00554371,  0.04991211,  0.09428051,
                    0.1386489 ,  0.1830173 ,  0.22738569,  0.27175409,
                    0.31612249],
             mask=False,
       fill_value=1e+20)

## Longitude

In [29]:
rain_lon[::8]

masked_array(data=[-10.95013957, -10.15013957,  -9.35013957,  -8.55013957,
                    -7.75013958,  -6.95013958,  -6.15013958,  -5.35013959,
                    -4.55013959,  -3.75013959,  -2.9501396 ,  -2.1501396 ,
                    -1.3501396 ,  -0.55013961,   0.24986039,   1.04986039,
                     1.84986038,   2.64986038],
             mask=False,
       fill_value=1e+20)

In [30]:
mf_lon[2:-1]

masked_array(data=[-1.08333e+01, -9.99995e+00, -9.16660e+00, -8.33325e+00,
                   -7.49990e+00, -6.66655e+00, -5.83320e+00, -4.99985e+00,
                   -4.16650e+00, -3.33315e+00, -2.49980e+00, -1.66645e+00,
                   -8.33100e-01,  2.50000e-04,  8.33600e-01,  1.66695e+00,
                    2.50030e+00,  3.33365e+00],
             mask=False,
       fill_value=1e+20)

In [36]:
( rain_lon[::8] - mf_lon[2:-1] ).mean()

-0.40031459240936296

In [35]:
(rain_lon[::8]  - mf_lon[1:-2] ).mean()

0.43303540759063697

In [42]:
rain_lon[::8] - mf_lon[2:-3]

masked_array(data=[-0.11683957, -0.05018957,  0.01646043,  0.08311042,
                    0.14976042,  0.21641042,  0.28306041,  0.34971041,
                    0.41636041,  0.4830104 ,  0.5496604 ,  0.6163104 ,
                    0.68296039,  0.74961039,  0.81626038,  0.88291038],
             mask=False,
       fill_value=1e+20)